<a href="https://colab.research.google.com/github/Alvan86/ISYE6501/blob/main/HW9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [272]:
pip install simpy

In [273]:
import simpy
import random
import statistics

In [278]:
lambda1 = 5  # Passenger arrival rate per minute
mu2 = 0.75  # Service rate in minute
min_scan = 0.5  # Min time at personal check
max_scan = 1.0  # Max time at personal check
num_checks = 2  # Number of ID/boarding-pass checkers
num_personal_queues = 3  # Number of personal-check queues
sim_duration = 720  # Total simulation time (minutes)
num_replications = 10  # Number of replications

In [279]:
# Initiate variables with 6 dimensions

avgCheckTime = [[0.0 for i in range(6)] for j in range(6)]
avgScanTime = [[0.0 for i in range(6)] for j in range(6)]
avgWaitTime = [[0.0 for i in range(6)] for j in range(6)]
avgSystemTime = [[0.0 for i in range(6)] for j in range(6)]

In [276]:
class AirportSecuritySystem:
    def __init__(self, env, i, j):
        self.env = env
        self.checker = simpy.Resource(env, i + num_checks)
        self.scanner = []
        for i in range(j + num_personal_queues):
          self.scanner.append(simpy.Resource(env,1))

    def id_check_process(self, passenger):
        yield self.env.timeout(random.expovariate(1.0/mu2))

    def personal_check_process(self, passenger):
        yield self.env.timeout(random.uniform(min_scan, max_scan))

def passenger_processing(env, name, s, i, j, pnum):
    global checkWait
    global scanWait
    global sysTime
    global totThrough
    timeArrive = env.now
    with s.checker.request() as request:
        yield request
        tIn = env.now  # Start check
        yield env.process(s.id_check_process(name))
        tOut = env.now # End check
        checkTime[pnum] = (tOut - tIn) # End check time - Start check time

    # Find the shortest personal-check queue
    shortest_queue = 0
    for k in range(1, j + num_personal_queues):
      if (len(s.scanner[k].queue) < len(s.scanner[shortest_queue].queue)):
        shortest_queue = k


    with s.scanner[shortest_queue].request() as request:
      yield request
      tIn = env.now # Start scan
      yield env.process(s.personal_check_process(name))
      tOut = env.now # End scan
      scanTime[pnum] = (tOut - tIn)  # End check time - Start check time

    timeLeaving = env.now
    sysTime[pnum] = (timeLeaving - timeArrive) # Total time in system for passenger
    totThrough += 1

In [277]:
def setup(env,i,j):
  k = 0
  s = AirportSecuritySystem(env,i,j)
  while True:
    yield env.timeout(random.expovariate(lambda1))
    k += 1
    env.process(passenger_processing(env,'Passenger %d' % k,s,i,j,k))

In [271]:
for i in range(6):
 for j in range(6):
  for k in range(num_replications):
      random.seed(k)

      env = simpy.Environment()

      totThrough = 0
      checkTime = [0.0] * int(lambda1 * sim_duration * 1.5)
      scanTime = [0.0] * int(lambda1 * sim_duration * 1.5)
      sysTime = [0.0] * int(lambda1 * sim_duration * 1.5)

      env.process(setup(env,i,j))
      env.run(until=sim_duration)

      avgSystemTime[i][j] += (sum(sysTime[1:totThrough]) / totThrough)
      avgCheckTime[i][j] += (sum(checkTime[1:totThrough]) / totThrough)
      avgScanTime[i][j] += (sum(scanTime[1:totThrough]) / totThrough)
      avgWaitTime[i][j] = (avgSystemTime[i][j] - avgCheckTime[i][j] - avgScanTime[i][j])

      avgSystemTime[i][j] /= num_replications
      avgCheckTime[i][j] /= num_replications
      avgScanTime[i][j] /= num_replications
      avgWaitTime[i][j] /= num_replications

      print('Number of ID/boarding pass checkers: %d Number of Personal-check queues: %d' % (i+2,j+3))
      print('Average system time = %.2f' % avgSystemTime[i][j])
      print('Average check time = %.2f' % avgCheckTime[i][j])
      print('Average scan time = %.2f' % avgScanTime[i][j])
      print('Average wait time = %.2f' % avgWaitTime[i][j])

Number of ID/boarding pass checkers: 2 Number of Personal-check queues: 3
Average system time = 38.16
Average check time = 0.08
Average scan time = 0.08
Average wait time = 38.00
Number of ID/boarding pass checkers: 2 Number of Personal-check queues: 3
Average system time = 39.28
Average check time = 0.08
Average scan time = 0.08
Average wait time = 39.11
Number of ID/boarding pass checkers: 2 Number of Personal-check queues: 3
Average system time = 37.30
Average check time = 0.08
Average scan time = 0.08
Average wait time = 37.13
Number of ID/boarding pass checkers: 2 Number of Personal-check queues: 3
Average system time = 37.10
Average check time = 0.08
Average scan time = 0.08
Average wait time = 36.93
Number of ID/boarding pass checkers: 2 Number of Personal-check queues: 3
Average system time = 37.56
Average check time = 0.08
Average scan time = 0.08
Average wait time = 37.39
Number of ID/boarding pass checkers: 2 Number of Personal-check queues: 3
Average system time = 37.95
Ave